# Training the model to run on Lichess data
## Some pre-requisites if running on Google Collab
If not running on Google collab do not run these next two cells!

In [ ]:
# Install the only dependency not available from collab directly
!pip install chess

# Get imported files from repo
!git clone -b lichess-neural-networks https://github.com/owenjaques/chessbot.git
!mv chessbot chessbot-repo
!mv chessbot-repo/neural_networks/chessbot .
!rm chessbot-repo -r

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
working_directory = '/content/gdrive/MyDrive/chessbot_weights/'
print(f'Saving to {working_directory}')

## If not running on Google Collab
Set the weights directory variable to wherever you would like data saved.

In [ ]:
!mkdir -p data
working_directory = './data/'

## Get the data
This compression format is really nice, so you can cancel this cell whenever you want and all the games that were downloaded will be maintained. In my experience 300Mb gets well over 100,000 games.

In [ ]:
!wget https://database.lichess.org/standard/lichess_db_standard_rated_2023-02.pgn.zst

## Decompress the Data

In [ ]:
!apt install zstd
!pzstd -d lichess_db_standard_rated_2023-02.pgn.zst

## Transform the Data
For this section we create a data generator which will play the games from disk then translate them into labelled model inputs which the model will train on.

In [ ]:
import chess
import chess.pgn
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from collections import deque

In [ ]:
from chessbot.model_input import ModelInput

class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, pgn_file, num_batches=128, name='training data', verbose=0):
        self.batch_size = batch_size
        self.num_batches = num_batches
        self.X_queue = deque()
        self.y_queue = deque()
        self.pgn = pgn_file
        self.n = batch_size * num_batches
        self.X = np.empty((self.n, 102))
        self.y = np.empty((self.n, 3), dtype=int)
        self.verbose = verbose
        self.name = name
        self.game_headers = []
        self.populate_Xy()

    def populate_Xy(self):
        # Plays games from the dataset and populates X and y
        
        i = 0
        while i < self.n:
            game = chess.pgn.read_game(self.pgn)
            if game is None:
                raise Exception('DataGenerator: Out of data to read from disk.')
    
            # Only train on game played to completion, that were not draws, and that have evaluations
            next_node = game.next()
            if not (game.headers['Termination'] == 'Normal' and game.headers['Result'] in ['1-0', '0-1'] and next_node and next_node.eval() != None):
                continue

            if game.headers in self.game_headers:
                raise Exception('Something has gone horribly wrong and we have a duplicate game.')
            self.game_headers.append(game.headers)

            try:
                # Generate the data from the game
                board = game.board()
                for node in game.mainline():
                    board.push(node.move)
                    self.X[i] = (ModelInput(board).get_input())
                    eval = node.eval().white().score(mate_score=10000)
                    self.y[i] = [eval <= -125, -125 < eval < 125, eval >= 125]
                    i += 1
                    
                    if self.verbose > 0:
                        print(f'\rGenerated {i}/{self.n} samples for {self.name}', end='')
            except:
                # There are a lot of reasons an exception could be thrown here, mostly stemming from bad data being parsed
                # from the pgn file. We just ignore these games and move on.
                pass

        if self.verbose > 0:
            print(f'\nNumber of losing positions: {np.sum(self.y[:, 0])}, Number of drawing positions: {np.sum(self.y[:, 1])}, Number of winning positions: {np.sum(self.y[:, 2])}')

        # For regression
        # if self.verbose > 1:
        #     _, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
        #     ax[0].hist(self.y, bins=50)
        #     ax[0].set_title(f'Labels for {self.name} (before normalising)')
        #     ax[0].set_xlabel('label (y)')
        #     ax[0].set_ylabel('no. of occurences in dataset')

        # Threshold to remove outliers and increase distribution of data
        # threshold_boundary = 1500
        # self.y[self.y > threshold_boundary] = threshold_boundary
        # self.y[self.y < -threshold_boundary] = -threshold_boundary

        # # Normalise in the range [0, 1]
        # self.y = (self.y - np.min(self.y)) / (np.max(self.y) - np.min(self.y))

        # if self.verbose > 1:
        #     ax[1].hist(self.y, bins=50)
        #     ax[1].set_title(f'Labels for {self.name} (after normalising)')
        #     ax[1].set_xlabel('label (y)')
        #     ax[1].set_ylabel('no. of occurences in dataset')
        #     plt.show()
        
    def on_epoch_end(self):
        self.populate_Xy()

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        # Returns one batch of data
        batch_idx_start = idx * self.batch_size
        batch_idx_end = idx * (self.batch_size + 1)
        return self.X[batch_idx_start:batch_idx_end], self.y[batch_idx_start:batch_idx_end]

## Our model
Set up your model being used here.

In [ ]:
model = keras.Sequential([
	keras.layers.Dense(512, activation='relu'),
	keras.layers.Dense(512, activation='relu'),
	keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
	metrics=['accuracy']
)

## Training the model
This next cell trains the model on the training data, then saves it to disk. Note multiple calls to this cell have crashed the notebook before due to high RAM usages.

In [ ]:
pgn_file = open('lichess_db_standard_rated_2023-02.pgn', 'r')
batch_size = 32
num_batches = 1024

training_data = DataGenerator(batch_size, pgn_file, num_batches, verbose=2)
validation_data = DataGenerator(batch_size, pgn_file, int(num_batches*0.2), name='validation data', verbose=2)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    restore_best_weights=True,
    patience=20,
    verbose=1)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=10,
    min_lr=0.000000000000001,
    verbose=1)

checkpoint = keras.callbacks.ModelCheckpoint(
    f'{working_directory}lichess_trained_model',
    monitor='val_loss',
    save_best_only=True)

tensorboard = keras.callbacks.TensorBoard(
    log_dir=f'{working_directory}/logs',
    write_graph=True,
    write_images=True,
    histogram_freq=1)

model.fit(
    training_data,
    epochs=128,
    shuffle=True,
    validation_data=validation_data,
    callbacks=[early_stopping, reduce_lr, checkpoint, tensorboard])

### Launch tensorboad

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /content/gdrive/MyDrive/chessbot_weights/logs/train

### Optionally load a previous model

In [ ]:
model = keras.models.load_model(f'{working_directory}/lichess_trained_model')

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = model.predict(validation_data.X)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(validation_data.y, axis=1)
cm = confusion_matrix(y_true, y_pred)

ax = sns.heatmap(cm, annot=True, fmt='d', xticklabels=['Losing', 'Drawing', 'Winning'], yticklabels=['Losing', 'Drawing', 'Winning'])
ax.set(xlabel='Predicted label', ylabel='True label')

## Why not play a game after all that training?

In [ ]:
import time
from IPython.display import clear_output
import chessbot.chessbot
from chessbot.chessbot import ChessBot

def play_game(model, exploration_rate=0.0, should_visualise=False):
	white = ChessBot(model, chess.WHITE, exploration_rate)
	black = ChessBot(model, chess.BLACK, exploration_rate)

	board = chess.Board()

	if should_visualise:
		display(board)

	while not board.is_game_over(claim_draw=True):
		board.push(black.move(board) if board.turn == chess.BLACK else white.move(board))

		if should_visualise:
			time.sleep(1)
			clear_output(wait=True)
			display(board)

	return board.outcome(claim_draw=True).result()
 
play_game(model, should_visualise=True)